## Read in the points, surfaces, tetrahedra, and UVCs

In [1]:
import numpy as np
import pandas as pd
import os
import re

class OpenCARPMeshReader:
    """
    Class to read openCARP mesh files (.pts, .elem, .surf) and UVC data (.csv)
    and convert them to NumPy arrays and pandas DataFrames.
    """
    
    def __init__(self, base_dir, prefix):
        """
        Initialize the reader with the directory and file prefix.
        
        Args:
            base_dir (str): Directory containing the mesh files
            prefix (str): Prefix for the mesh files (e.g., 'heart_instance_001')
        """
        self.base_dir = base_dir
        self.prefix = prefix
        
        # Set file paths
        self.pts_file = os.path.join(base_dir, f"{prefix}.pts")
        self.elem_file = os.path.join(base_dir, f"{prefix}.elem")
        self.surf_file = os.path.join(base_dir, f"{prefix}.surf")
        self.uvc_file = os.path.join(base_dir, f"UVC_{prefix}.csv")
        
        # Storage for mesh components
        self.points = None
        self.tetrahedra = None
        self.triangles = None
        self.triangle_regions = None
        self.tetrahedra_regions = None
        self.uvc_data = None
    
    def read_points(self):
        """
        Read the .pts file and return an array of points with dimension (n_points x 3).
        
        Returns:
            numpy.ndarray: Array of 3D points
        """
        print(f"Reading points from {self.pts_file}")
        
        with open(self.pts_file, 'r') as f:
            # Read the number of points from the first line
            n_points = int(f.readline().strip())
            
            # Initialize the points array
            points = np.zeros((n_points, 3))
            
            # Read each point
            for i in range(n_points):
                line = f.readline().strip()
                # Parse x, y, z coordinates
                x, y, z = map(float, line.split())
                points[i] = [x, y, z]
        
        self.points = points
        print(f"Loaded {n_points} points with shape {points.shape}")
        return points
    
    def read_tetrahedra(self):
        """
        Read the .elem file and return an array of tetrahedra with dimension (n_tetrahedra x 4).
        Each tetrahedron contains indices to the points array.
        
        Returns:
            numpy.ndarray: Array of tetrahedra indices
            numpy.ndarray: Array of region IDs for each tetrahedron
        """
        print(f"Reading tetrahedra from {self.elem_file}")
        
        tetrahedra = []
        tetrahedra_regions = []
        
        with open(self.elem_file, 'r') as f:
            # Read the number of elements from the first line
            n_elements = int(f.readline().strip())
            
            # Read each tetrahedron
            for i in range(n_elements):
                line = f.readline().strip()
                
                # Parse the line for a tetrahedron (Tt format)
                if line.startswith('Tt'):
                    parts = line.split()
                    
                    # Extract the 4 vertex indices
                    indices = [int(parts[1]), int(parts[2]), int(parts[3]), int(parts[4])]
                    
                    # Extract region ID (if available)
                    region_id = int(parts[5]) if len(parts) > 5 else 0
                    
                    tetrahedra.append(indices)
                    tetrahedra_regions.append(region_id)
        
        # Convert to NumPy arrays
        tetrahedra = np.array(tetrahedra)
        tetrahedra_regions = np.array(tetrahedra_regions)
        
        self.tetrahedra = tetrahedra
        self.tetrahedra_regions = tetrahedra_regions
        
        print(f"Loaded {len(tetrahedra)} tetrahedra with shape {tetrahedra.shape}")
        return tetrahedra, tetrahedra_regions
    
    def read_triangles(self):
        """
        Read the .surf file and return an array of triangles with dimension (n_triangles x 3).
        Each triangle contains indices to the points array.
        
        Returns:
            numpy.ndarray: Array of triangle indices
            numpy.ndarray: Array of region IDs for each triangle
        """
        print(f"Reading triangles from {self.surf_file}")
        
        triangles = []
        triangle_regions = []
        
        with open(self.surf_file, 'r') as f:
            lines = f.readlines()
            
            i = 0
            while i < len(lines):
                line = lines[i].strip()
                i += 1
                
                # Check if this line defines a region
                region_match = re.match(r"(\d+)(?:\s+Reg\s+(\d+))?", line)
                if region_match:
                    n_triangles_in_region = int(region_match.group(1))
                    
                    if region_match.group(2):
                        region_id = int(region_match.group(2))
                    else:
                        region_id = -1
                    
                    # Read triangles for this region
                    for j in range(n_triangles_in_region):
                        if i < len(lines):
                            tri_line = lines[i].strip()
                            i += 1
                            
                            # Parse triangle (Tr format)
                            if tri_line.startswith('Tr'):
                                parts = tri_line.split()
                                # Extract the 3 vertex indices
                                indices = [int(parts[1]), int(parts[2]), int(parts[3])]
                                
                                triangles.append(indices)
                                triangle_regions.append(region_id)
        
        # Convert to NumPy arrays
        triangles = np.array(triangles)
        triangle_regions = np.array(triangle_regions)
        
        self.triangles = triangles
        self.triangle_regions = triangle_regions
        
        print(f"Loaded {len(triangles)} triangles with shape {triangles.shape}")
        return triangles, triangle_regions
    
    def read_uvc_data(self):
        """
        Read the UVC data from the CSV file into a pandas DataFrame.
        
        Returns:
            pandas.DataFrame: DataFrame containing UVC data
        """
        if os.path.exists(self.uvc_file):
            print(f"Reading UVC data from {self.uvc_file}")
            
            # Read the CSV file into a pandas DataFrame
            uvc_data = pd.read_csv(self.uvc_file)
            
            self.uvc_data = uvc_data
            
            print(f"Loaded UVC data with shape {uvc_data.shape}")
            print(f"UVC columns: {uvc_data.columns.tolist()}")
            
            return uvc_data
        else:
            print(f"UVC file not found: {self.uvc_file}")
            return None
    
    def read_all(self):
        """
        Read all mesh files and UVC data.
        
        Returns:
            tuple: (points, tetrahedra, tetrahedra_regions, triangles, triangle_regions, uvc_data)
        """
        points = self.read_points()
        tetrahedra, tetrahedra_regions = self.read_tetrahedra()
        triangles, triangle_regions = self.read_triangles()
        uvc_data = self.read_uvc_data()
        
        return (points, tetrahedra, tetrahedra_regions, triangles, triangle_regions, uvc_data)

data_path = "/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/"
output_prefix = "heart_instance_001_lowres"
reader = OpenCARPMeshReader(data_path, output_prefix)
points, tetrahedra, tetrahedra_regions, triangles, triangle_regions, uvc_data = reader.read_all()

Reading points from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.pts
Loaded 110822 points with shape (110822, 3)
Reading tetrahedra from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.elem
Loaded 567173 tetrahedra with shape (567173, 4)
Reading triangles from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.surf
Loaded 77128 triangles with shape (77128, 3)
Reading UVC data from /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/UVC_heart_instance_001_lowres.csv
Loaded UVC data with shape (110822, 6)
UVC columns: ['tv', 'tm', 'rtSin', 'rtCos', 'rt', 'ab']


In [2]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as splinalg
import pyvista as pv

def compute_normed_gradients(points, phi_transmural, phi_longitudinal, phi_circumferential):
    """Compute normalized gradient of phi."""
    from scipy.spatial import KDTree
    
    tree = KDTree(points)
    grad_phi_transmural = np.zeros_like(points)
    grad_phi_longitudinal = np.zeros_like(points)
    grad_phi_circumferential = np.zeros_like(points)
    
    for i, p in enumerate(points):
        _, neighbors = tree.query(p, k=20)  # Find nearby points
        
        neighbor_points = points[neighbors]
        
        neighbor_phi_transmural = phi_transmural[neighbors]
        grad = np.linalg.lstsq(neighbor_points - p, neighbor_phi_transmural - phi_transmural[i], rcond=None)[0]
        grad_phi_transmural[i] = grad / (1e-9 + np.linalg.norm(grad))  # Normalize

        neighbor_phi_longitudinal = phi_longitudinal[neighbors]
        grad = np.linalg.lstsq(neighbor_points - p, neighbor_phi_longitudinal - phi_longitudinal[i], rcond=None)[0]
        grad_phi_longitudinal[i] = grad / (1e-9 + np.linalg.norm(grad))  # Normalize

        neighbor_phi_circumferential = phi_circumferential[neighbors]
        grad = np.linalg.lstsq(neighbor_points - p, neighbor_phi_circumferential - phi_circumferential[i], rcond=None)[0]
        grad_phi_circumferential[i] = grad / (1e-9 + np.linalg.norm(grad))  # Normalize
    
    return grad_phi_transmural, grad_phi_longitudinal, grad_phi_circumferential

phi_transmural = np.array(uvc_data['tm'])
phi_longitudinal = np.array(uvc_data['ab'])
phi_circumferential = np.array(uvc_data["rt"])
TransmuralField, LongitudinalField, CircumferentialField = compute_normed_gradients(points, -phi_transmural, -phi_longitudinal, phi_circumferential)

In [3]:
import numpy as np
import pandas as pd
import pyvista as pv
from pyvista import themes
import os
import random

def visualize_vector_fields(points, triangles, triangle_regions, 
                           transmural_field, longitudinal_field, circumferential_field, 
                           subsample_factor=20, glyph_scale=0.005):
    """
    Create side-by-side visualizations of the vector fields on epicardium and endocardium.
    
    Left plot: Epicardium (triangle_regions == 2)
    Right plot: Endocardium (triangle_regions == 3 or 4)
    """
    # Set up a nice theme for visualization
    my_theme = themes.DarkTheme()
    my_theme.lighting = True
    my_theme.show_edges = False
    my_theme.background = 'white'
    my_theme.window_size = [1600, 800]
    pv.set_plot_theme(my_theme)
    
    print(f"Original points: {len(points)}")
    print(f"Original triangles: {len(triangles)}")
    
    # Prepare triangle cells for PyVista format
    triangle_cells = np.column_stack(
        (np.full(len(triangles), 3), triangles)
    ).flatten()
    
    # Create full mesh
    full_mesh = pv.PolyData(points, triangle_cells)
    
    # Create a plotter with two side-by-side viewports
    plotter = pv.Plotter(shape=(1, 2))
    
    # Filter triangles by region
    epicardium_triangles = triangles[np.logical_or(triangle_regions == 2, triangle_regions == -1)]
    endocardium_triangles = triangles[np.logical_or(triangle_regions == 3, np.logical_or(triangle_regions == 4, triangle_regions == -1))]
    
    print(f"Epicardium triangles: {len(epicardium_triangles)}")
    print(f"Endocardium triangles: {len(endocardium_triangles)}")
    
    # Create separate meshes for each surface
    epicardium_cells = np.column_stack(
        (np.full(len(epicardium_triangles), 3), epicardium_triangles)
    ).flatten()
    
    endocardium_cells = np.column_stack(
        (np.full(len(endocardium_triangles), 3), endocardium_triangles)
    ).flatten()
    
    epicardium_mesh = pv.PolyData(points, epicardium_cells)
    endocardium_mesh = pv.PolyData(points, endocardium_cells)
    
    # Function to sample points from a mesh
    def create_sampled_vectors(mesh, field_names=["transmural", "longitudinal", "circumferential"]):
        # Extract unique points from this mesh
        unique_point_ids = np.unique(mesh.faces.reshape(-1, 4)[:, 1:].flatten())
        
        # Further sample to reduce number of arrows
        n_sample = min(len(unique_point_ids), len(unique_point_ids) // subsample_factor)
        sample_indices = sorted(random.sample(list(unique_point_ids), n_sample))
        
        # Create a point cloud with the sampled points
        sampled_points = points[sample_indices]
        point_cloud = pv.PolyData(sampled_points)
        
        # Add vector data
        point_cloud["transmural"] = transmural_field[sample_indices]
        point_cloud["longitudinal"] = longitudinal_field[sample_indices]
        point_cloud["circumferential"] = circumferential_field[sample_indices]
        
        return point_cloud, sample_indices
    
    # Create sampled vector fields for each surface
    epicardium_vectors, epi_indices = create_sampled_vectors(epicardium_mesh)
    endocardium_vectors, endo_indices = create_sampled_vectors(endocardium_mesh)
    
    print(f"Sampled epicardium points: {epicardium_vectors.n_points}")
    print(f"Sampled endocardium points: {endocardium_vectors.n_points}")
    
    # Left viewport: Epicardium
    plotter.subplot(0, 0)
    plotter.add_text("Epicardium Vector Fields", font_size=16, color="black")
    
    # Add epicardium surface
    plotter.add_mesh(epicardium_mesh, color='lightgray', opacity=0.8)
    
    # Add vector fields for epicardium
    transmural_arrows = epicardium_vectors.glyph(
        orient="transmural",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(transmural_arrows, color="red", label="Transmural")
    
    longitudinal_arrows = epicardium_vectors.glyph(
        orient="longitudinal",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(longitudinal_arrows, color="green", label="Longitudinal")
    
    circumferential_arrows = epicardium_vectors.glyph(
        orient="circumferential",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(circumferential_arrows, color="blue", label="Circumferential")
    
    # Add legend
    plotter.add_legend(size=(0.2, 0.2), loc='upper right')
    
    # Right viewport: Endocardium
    plotter.subplot(0, 1)
    plotter.add_text("Endocardium Vector Fields", font_size=16, color="black")
    
    # Add endocardium surface
    plotter.add_mesh(endocardium_mesh, color='lightgray', opacity=0.8)
    
    # Add vector fields for endocardium
    transmural_arrows = endocardium_vectors.glyph(
        orient="transmural",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(transmural_arrows, color="red", label="Transmural")
    
    longitudinal_arrows = endocardium_vectors.glyph(
        orient="longitudinal",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(longitudinal_arrows, color="green", label="Longitudinal")
    
    circumferential_arrows = endocardium_vectors.glyph(
        orient="circumferential",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(circumferential_arrows, color="blue", label="Circumferential")
    
    # Add legend
    plotter.add_legend(size=(0.2, 0.2), loc='upper right')
    
    # Link the cameras between the two viewports
    plotter.link_views()
    
    # Set camera position
    plotter.camera_position = 'xz'
    plotter.camera.zoom(1.2)
    
    # Show the plot
    plotter.show()

    plotter.screenshot('../figures/cardial_coordinates.png')

# Visualize the fields
visualize_vector_fields(
    points, triangles, triangle_regions,
    TransmuralField, LongitudinalField, CircumferentialField, subsample_factor=100, glyph_scale=4
)

Original points: 110822
Original triangles: 77128
Epicardium triangles: 35848
Endocardium triangles: 39581
Sampled epicardium points: 180
Sampled endocardium points: 199


Widget(value='<iframe src="http://localhost:63147/index.html?ui=P_0x1422263b0_0&reconnect=auto" class="pyvista…

In [4]:
import numpy as np

def compute_fiber_sheet_directions(TransmuralField, LongitudinalField, CircumferentialField, phi_transmural):
    """
    Compute rule-based cardiac fiber and sheet directions.
    
    The fiber angles rotate from +60° at the endocardium (phi=0) to -60° at the epicardium (phi=1).
    The sheet angles rotate from -65° at the endocardium to +25° at the epicardium.
    """
    # Number of points
    n_points = len(TransmuralField)
    
    # Initialize output arrays
    fiber_directions = np.zeros_like(TransmuralField)
    sheet_directions = np.zeros_like(TransmuralField)
    sheet_normal_directions = np.zeros_like(TransmuralField)
    
    # Fiber angle parameters (in radians)
    endo_fiber_angle = np.radians(60.0)  # +60° at endocardium
    epi_fiber_angle = np.radians(-60.0)  # -60° at epicardium
    
    # Sheet angle parameters (in radians)
    endo_sheet_angle = np.radians(-65.0)  # -65° at endocardium
    epi_sheet_angle = np.radians(25.0)    # +25° at epicardium
    
    for i in range(n_points):
        # Normalize local coordinate system vectors
        t_vec = TransmuralField[i] / np.linalg.norm(TransmuralField[i])
        l_vec = LongitudinalField[i] / np.linalg.norm(LongitudinalField[i])
        c_vec = CircumferentialField[i] / np.linalg.norm(CircumferentialField[i])
        
        # Interpolate fiber angle based on transmural position (linear interpolation)
        phi = phi_transmural[i]
        fiber_angle = endo_fiber_angle * (1 - phi) + epi_fiber_angle * phi
        
        # Interpolate sheet angle based on transmural position (linear interpolation)
        sheet_angle = endo_sheet_angle * (1 - phi) + epi_sheet_angle * phi
        
        # Compute fiber direction by rotating the circumferential vector toward the longitudinal direction
        # Rotation around transmural axis
        fiber_dir = c_vec * np.cos(fiber_angle) + l_vec * np.sin(fiber_angle)
        fiber_dir = fiber_dir / np.linalg.norm(fiber_dir)
        
        # IMPROVED SHEET CALCULATION:
        # First, define sheet direction as a rotation of the transmural vector in the transmural-longitudinal plane
        # This rotation is around an axis perpendicular to both transmural and longitudinal vectors (i.e., circumferential)
        sheet_dir = t_vec * np.cos(sheet_angle) + l_vec * np.sin(sheet_angle)
        
        # Make sheet_dir orthogonal to fiber_dir using Gram-Schmidt process
        # Project out any component of sheet_dir that's parallel to fiber_dir
        fiber_component = np.dot(sheet_dir, fiber_dir) * fiber_dir
        sheet_dir = sheet_dir - fiber_component
        sheet_dir = sheet_dir / np.linalg.norm(sheet_dir)
        
        # Compute sheet normal as cross product to ensure perfect orthogonality
        sheet_normal = np.cross(fiber_dir, sheet_dir)
        sheet_normal = sheet_normal / np.linalg.norm(sheet_normal)
        
        # Store the results
        fiber_directions[i] = fiber_dir
        sheet_directions[i] = sheet_dir
        sheet_normal_directions[i] = sheet_normal
    
    return fiber_directions, sheet_directions, sheet_normal_directions

def verify_orthogonality(fiber_directions, sheet_directions, sheet_normal_directions):
    """
    Verify that the fiber, sheet, and sheet normal directions form an orthogonal basis.
    
    Parameters:
    -----------
    fiber_directions : numpy.ndarray
        Fiber direction vectors, shape (n_points, 3)
    sheet_directions : numpy.ndarray
        Sheet direction vectors, shape (n_points, 3)
    sheet_normal_directions : numpy.ndarray
        Sheet normal direction vectors, shape (n_points, 3)
        
    Returns:
    --------
    tuple
        (max_f_s_dot, max_f_n_dot, max_s_n_dot) maximum absolute dot products between vectors
    """
    n_points = len(fiber_directions)
    
    max_f_s_dot = 0
    max_f_n_dot = 0
    max_s_n_dot = 0
    
    for i in range(n_points):
        f = fiber_directions[i]
        s = sheet_directions[i]
        n = sheet_normal_directions[i]
        
        # Compute dot products (should be close to 0 for orthogonal vectors)
        f_s_dot = abs(np.dot(f, s))
        f_n_dot = abs(np.dot(f, n))
        s_n_dot = abs(np.dot(s, n))
        
        # Track maximum values
        max_f_s_dot = max(max_f_s_dot, f_s_dot)
        max_f_n_dot = max(max_f_n_dot, f_n_dot)
        max_s_n_dot = max(max_s_n_dot, s_n_dot)
    
    return max_f_s_dot, max_f_n_dot, max_s_n_dot

def validate_right_handed_system(fiber_directions, sheet_directions, sheet_normal_directions):
    """
    Validate that the basis vectors form a right-handed orthonormal system.
    
    In a right-handed system, fiber × sheet = sheet_normal
    
    Parameters:
    -----------
    fiber_directions : numpy.ndarray
        Fiber direction vectors, shape (n_points, 3)
    sheet_directions : numpy.ndarray
        Sheet direction vectors, shape (n_points, 3)
    sheet_normal_directions : numpy.ndarray
        Sheet normal direction vectors, shape (n_points, 3)
        
    Returns:
    --------
    float
        Minimum dot product between computed cross product and sheet normal
        (should be close to 1 for consistent right-handed systems)
    """
    n_points = len(fiber_directions)
    min_alignment = 1.0
    
    for i in range(n_points):
        f = fiber_directions[i]
        s = sheet_directions[i]
        n = sheet_normal_directions[i]
        
        # Compute cross product
        expected_normal = np.cross(f, s)
        expected_normal = expected_normal / np.linalg.norm(expected_normal)
        
        # Check alignment with provided normal (dot product should be close to 1)
        alignment = np.dot(expected_normal, n)
        min_alignment = min(min_alignment, alignment)
    
    return min_alignment

# Example usage:
fiber_dirs, sheet_dirs, sheet_normal_dirs = compute_fiber_sheet_directions(
    TransmuralField, LongitudinalField, CircumferentialField, phi_transmural)

# Verify orthogonality
max_dots = verify_orthogonality(fiber_dirs, sheet_dirs, sheet_normal_dirs)
print(f"Maximum dot products: {max_dots}")

# Validate right-handed system
min_alignment = validate_right_handed_system(fiber_dirs, sheet_dirs, sheet_normal_dirs)
print(f"Minimum right-hand system alignment: {min_alignment}")

Maximum dot products: (np.float64(1.1879386363489175e-14), np.float64(2.220446049250313e-16), np.float64(1.8735013540549517e-16))
Minimum right-hand system alignment: 0.9999999999999996


In [5]:
import numpy as np
import pandas as pd
import pyvista as pv
from pyvista import themes
import os
import random

def visualize_fibers(points, triangles, triangle_regions, fiber_dirs, 
                    subsample_factor=20, glyph_scale=0.005):
    """
    Create side-by-side visualizations of the fiber directions on epicardium and endocardium.
    
    Left plot: Endocardium (triangle_regions == 3 or 4) in red
    Right plot: Epicardium (triangle_regions == 2) in blue
    
    Parameters:
    -----------
    points : numpy.ndarray
        Array of 3D coordinates of the mesh vertices
    triangles : numpy.ndarray
        Array of triangle indices (each row contains 3 indices)
    triangle_regions : numpy.ndarray
        Array indicating the region each triangle belongs to
        (2 for epicardium, 3 or 4 for endocardium)
    fiber_dirs : numpy.ndarray
        Array of fiber direction vectors for each point
    subsample_factor : int, optional
        Factor by which to subsample points for vector visualization
    glyph_scale : float, optional
        Scale factor for the vector glyphs
    """
    # Set up a nice theme for visualization
    my_theme = themes.DarkTheme()
    my_theme.lighting = True
    my_theme.show_edges = False
    my_theme.background = 'white'
    my_theme.window_size = [1600, 800]
    pv.set_plot_theme(my_theme)
    
    print(f"Original points: {len(points)}")
    print(f"Original triangles: {len(triangles)}")
    
    # Prepare triangle cells for PyVista format
    triangle_cells = np.column_stack(
        (np.full(len(triangles), 3), triangles)
    ).flatten()
    
    # Create full mesh
    full_mesh = pv.PolyData(points, triangle_cells)
    
    # Create a plotter with two side-by-side viewports
    plotter = pv.Plotter(shape=(1, 2))
    
    # Filter triangles by region
    epicardium_triangles = triangles[np.logical_or(triangle_regions == 2, triangle_regions == -1)]
    endocardium_triangles = triangles[np.logical_or(triangle_regions == 3, np.logical_or(triangle_regions == 4, triangle_regions == -1))]
    
    print(f"Epicardium triangles: {len(epicardium_triangles)}")
    print(f"Endocardium triangles: {len(endocardium_triangles)}")
    
    # Create separate meshes for each surface
    epicardium_cells = np.column_stack(
        (np.full(len(epicardium_triangles), 3), epicardium_triangles)
    ).flatten()
    
    endocardium_cells = np.column_stack(
        (np.full(len(endocardium_triangles), 3), endocardium_triangles)
    ).flatten()
    
    epicardium_mesh = pv.PolyData(points, epicardium_cells)
    endocardium_mesh = pv.PolyData(points, endocardium_cells)
    
    # Function to sample points from a mesh
    def create_sampled_vectors(mesh):
        # Extract unique points from this mesh
        unique_point_ids = np.unique(mesh.faces.reshape(-1, 4)[:, 1:].flatten())
        
        # Further sample to reduce number of arrows
        n_sample = min(len(unique_point_ids), len(unique_point_ids) // subsample_factor)
        sample_indices = sorted(random.sample(list(unique_point_ids), n_sample))
        
        # Create a point cloud with the sampled points
        sampled_points = points[sample_indices]
        point_cloud = pv.PolyData(sampled_points)
        
        # Add vector data
        point_cloud["fibers"] = fiber_dirs[sample_indices]
        
        return point_cloud, sample_indices
    
    # Create sampled vector fields for each surface
    epicardium_vectors, epi_indices = create_sampled_vectors(epicardium_mesh)
    endocardium_vectors, endo_indices = create_sampled_vectors(endocardium_mesh)
    
    print(f"Sampled epicardium points: {epicardium_vectors.n_points}")
    print(f"Sampled endocardium points: {endocardium_vectors.n_points}")
    
    # Left viewport: Endocardium (in red)
    plotter.subplot(0, 0)
    plotter.add_text("Endocardium Fiber Directions", font_size=16, color="black")
    
    # Add endocardium surface
    plotter.add_mesh(endocardium_mesh, color='mistyrose', opacity=0.8)
    
    # Add fiber directions for endocardium
    fiber_arrows = endocardium_vectors.glyph(
        orient="fibers",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(fiber_arrows, color="red", label="Fiber Direction")
    
    # Add legend
    plotter.add_legend(size=(0.2, 0.2), loc='upper right')
    
    # Right viewport: Epicardium (in blue)
    plotter.subplot(0, 1)
    plotter.add_text("Epicardium Fiber Directions", font_size=16, color="black")
    
    # Add epicardium surface
    plotter.add_mesh(epicardium_mesh, color='lightblue', opacity=0.8)
    
    # Add fiber directions for epicardium
    fiber_arrows = epicardium_vectors.glyph(
        orient="fibers",
        scale=False,
        factor=glyph_scale
    )
    plotter.add_mesh(fiber_arrows, color="blue", label="Fiber Direction")
    
    # Add legend
    plotter.add_legend(size=(0.2, 0.2), loc='upper right')
    
    # Link the cameras between the two viewports
    plotter.link_views()
    
    # Set camera position
    plotter.camera_position = 'xz'
    plotter.camera.zoom(1.2)
    
    # Show the plot
    plotter.show()

    plotter.screenshot('../figures/fiber_directions.png')    

visualize_fibers(
    points, triangles, triangle_regions,
    fiber_dirs, subsample_factor=50, glyph_scale=5
)

Original points: 110822
Original triangles: 77128
Epicardium triangles: 35848
Endocardium triangles: 39581
Sampled epicardium points: 361
Sampled endocardium points: 399


Widget(value='<iframe src="http://localhost:63147/index.html?ui=P_0x137d2e560_1&reconnect=auto" class="pyvista…

In [34]:
import numpy as np
from tqdm import tqdm  # For progress bar

def average_unit_vector(vectors):
    """
    Calculate the average unit vector from a set of vectors.
    """
    average_vector = np.mean(vectors, axis=0)
    norm = np.linalg.norm(average_vector)
    
    # Avoid division by zero
    if norm < 1e-10:
        return np.array([1.0, 0.0, 0.0])  # Default to x-axis if vectors cancel out
    
    return average_vector / norm

def precompute_element_directions(fiber_dirs, sheet_dirs, sheet_normal_dirs, tetrahedra):
    """
    Precompute the averaged fiber, sheet and sheet normal directions for all elements
    """
    n_elements = len(tetrahedra)
    element_fiber_dirs = np.zeros((n_elements, 3))
    element_sheet_dirs = np.zeros((n_elements, 3))
    element_normal_dirs = np.zeros((n_elements, 3))
    
    # Use vectorized operations where possible
    for i in tqdm(range(n_elements), desc="Precomputing element directions"):
        # Get the nodes for this element
        tet_nodes = tetrahedra[i]
        
        # Average the directions for this element
        element_fiber_dirs[i] = average_unit_vector(fiber_dirs[tet_nodes])
        element_sheet_dirs[i] = average_unit_vector(sheet_dirs[tet_nodes])
        element_normal_dirs[i] = average_unit_vector(sheet_normal_dirs[tet_nodes])
    
    return element_fiber_dirs, element_sheet_dirs, element_normal_dirs

def write_lon_file(filename, fiber_directions, sheet_directions, sheet_normal_directions, tetrahedra, batch_size=10000):
    """
    Write fiber, sheet, and sheet normal directions to a .lon file for openCARP.
    """
    n_elements = len(tetrahedra)
    
    print(f"Processing {n_elements} elements for .lon file...")
    
    # Precompute all element directions at once
    element_fiber_dirs, element_sheet_dirs, element_normal_dirs = precompute_element_directions(
        fiber_directions, sheet_directions, sheet_normal_directions, tetrahedra
    )
    
    # Verify orthogonality of precomputed directions
    print("Verifying orthogonality of element directions...")
    max_dot_f_s = np.max(np.abs(np.sum(element_fiber_dirs * element_sheet_dirs, axis=1)))
    max_dot_f_n = np.max(np.abs(np.sum(element_fiber_dirs * element_normal_dirs, axis=1)))
    max_dot_s_n = np.max(np.abs(np.sum(element_sheet_dirs * element_normal_dirs, axis=1)))
    
    print(f"Maximum dot products: fiber·sheet={max_dot_f_s:.6f}, fiber·normal={max_dot_f_n:.6f}, sheet·normal={max_dot_s_n:.6f}")
    
    print(f"Writing to {filename}...")
    
    # Write to file in batches
    with open(filename, 'w') as f:
        # Header: 3 for fiber, sheet, and sheet-normal directions
        f.write('2\n')
        
        # Write in batches to avoid memory issues
        for i in tqdm(range(0, n_elements, batch_size), desc="Writing .lon file"):
            batch_end = min(i + batch_size, n_elements)
            batch_elements = range(i, batch_end)
            
            # Create a single string for the batch
            lines = []
            for j in batch_elements:
                f_vec = element_fiber_dirs[j]
                s_vec = element_sheet_dirs[j]
                line = f"{f_vec[0]:.8f} {f_vec[1]:.8f} {f_vec[2]:.8f} " + \
                       f"{s_vec[0]:.8f} {s_vec[1]:.8f} {s_vec[2]:.8f} "
                lines.append(line)
            
            # Write the batch
            f.write('\n'.join(lines) + '\n')
    
    print(f"Successfully wrote fiber orientations to {filename}")

write_lon_file(
    f"{data_path}{output_prefix}.lon", 
    fiber_dirs, 
    sheet_dirs, 
    sheet_normal_dirs,  # Now including sheet normal directions
    tetrahedra
)

Processing 260386 elements for .lon file...


Precomputing element directions:   0%|          | 0/260386 [00:00<?, ?it/s]

Precomputing element directions: 100%|██████████| 260386/260386 [00:04<00:00, 60437.20it/s]


Verifying orthogonality of element directions...
Maximum dot products: fiber·sheet=0.999433, fiber·normal=0.999891, sheet·normal=0.998845
Writing to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.lon...


Writing .lon file: 100%|██████████| 27/27 [00:00<00:00, 46.63it/s]

Successfully wrote fiber orientations to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.lon


In [35]:
import numpy as np
from tqdm import tqdm

def tag_fast_conducting_endocardium(
    points, 
    tetrahedra, 
    triangles, 
    triangle_regions, 
    phi_longitudinal, 
    output_file
):
    """
    Tag tetrahedra for fast conducting endocardium based on:
    1. Points with phi_longitudinal between 0.1 and 0.9
    2. Points that are part of triangles with triangle_regions == 3 or 4
    """
    # Step 1: Identify points that satisfy phi_longitudinal criterion
    print("Identifying points with phi_longitudinal between 0.1 and 0.9...")
    longitudinal_mask = (phi_longitudinal >= 0.1) & (phi_longitudinal <= 0.9)
    valid_longitudinal_points = set(np.where(longitudinal_mask)[0])
    print(f"Found {len(valid_longitudinal_points)} points with valid longitudinal coordinate")
    
    # Step 2: Identify points that are part of endocardial triangles (regions 3 or 4)
    print("Identifying endocardial points...")
    endocardial_triangles = np.where((triangle_regions == 3) | (triangle_regions == 4))[0]
    endocardial_points = set()
    
    for tri_idx in endocardial_triangles:
        for point_idx in triangles[tri_idx]:
            endocardial_points.add(point_idx)
    
    print(f"Found {len(endocardial_points)} points on the endocardium")
    
    # Step 3: Find intersection of both sets
    valid_points = valid_longitudinal_points.intersection(endocardial_points)
    print(f"Found {len(valid_points)} points that satisfy both criteria")
    
    # Step 4: Mark tetrahedra containing valid points
    print("Tagging tetrahedra...")
    n_tetrahedra = tetrahedra.shape[0]
    tetrahedra_tags = np.zeros(n_tetrahedra, dtype=int)
    
    # Convert to set for faster lookups
    valid_points_set = set(valid_points)
    
    # Process in batches to avoid memory issues
    batch_size = 10000
    for batch_start in tqdm(range(0, n_tetrahedra, batch_size)):
        batch_end = min(batch_start + batch_size, n_tetrahedra)
        batch_tetrahedra = tetrahedra[batch_start:batch_end]
        
        # Check each tetrahedron in the batch
        for i, tet in enumerate(batch_tetrahedra):
            # If any point in the tetrahedron is valid, tag it as 1
            if any(point_idx in valid_points_set for point_idx in tet):
                tetrahedra_tags[batch_start + i] = 1
    
    num_tagged = np.sum(tetrahedra_tags == 1)
    print(f"Tagged {num_tagged} tetrahedra ({(num_tagged/n_tetrahedra)*100:.2f}%) as fast conducting endocardium")
    
    # Step 5: Write the modified .elem file
    print(f"Writing modified element file to {output_file}...")
    with open(output_file, 'w') as fout:
        # Write header (number of tetrahedra)
        fout.write(f"{n_tetrahedra}\n")
        
        # Process each tetrahedron and write to file
        for i in tqdm(range(n_tetrahedra)):
            tet = tetrahedra[i]
            # Format: Tt node1 node2 node3 node4 tag
            line = f"Tt {tet[0]} {tet[1]} {tet[2]} {tet[3]} {tetrahedra_tags[i]}"
            fout.write(f"{line}\n")
    
    print(f"Successfully wrote modified element file to {output_file}")

tag_fast_conducting_endocardium(
    points, 
    tetrahedra, 
    triangles, 
    triangle_regions, 
    phi_longitudinal, 
    f"{data_path}{output_prefix}.elem"
)

Identifying points with phi_longitudinal between 0.1 and 0.9...
Found 45296 points with valid longitudinal coordinate
Identifying endocardial points...
Found 11843 points on the endocardium
Found 9917 points that satisfy both criteria
Tagging tetrahedra...


100%|██████████| 27/27 [00:00<00:00, 148.31it/s]


Tagged 72064 tetrahedra (27.68%) as fast conducting endocardium
Writing modified element file to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.elem...


100%|██████████| 260386/260386 [00:00<00:00, 816211.29it/s]

Successfully wrote modified element file to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.elem


In [26]:
import numpy as np
from scipy.cluster.vq import kmeans2

def filter_fascicular_sites_closer_to_LV(points, is_fascicular_site, septal_normal, LV_center, RV_center):
    """
    Filter fascicular sites to keep only those closer to the left ventricle.
    """
    # Get indices and coordinates of all fascicular sites
    fascicular_indices = np.where(is_fascicular_site)[0]
    fascicular_points = points[fascicular_indices]
    
    print(f"Total fascicular sites: {len(fascicular_indices)}")
    
    if len(fascicular_indices) == 0:
        print("No fascicular sites found.")
        return is_fascicular_site
    
    # If there's only one cluster, check if it's closer to LV or RV
    if len(fascicular_indices) < 5:  # Not enough points for reliable clustering
        # Compute signed distance to the plane separating LV and RV
        # The plane passes through the middle point between LV and RV centers
        # with septal_normal as its normal
        mid_point = (LV_center + RV_center) / 2
        
        # For each fascicular point, calculate its position relative to the plane
        # Positive values mean the point is on the LV side
        distances = np.dot(fascicular_points - mid_point, septal_normal)
        
        if np.all(distances > 0) or np.all(distances < 0):
            # All points are on the same side
            is_lv_side = np.mean(distances) > 0
            print(f"All fascicular sites are on the {'LV' if is_lv_side else 'RV'} side.")
            if not is_lv_side:
                # If all points are on RV side, return empty array
                return np.zeros_like(is_fascicular_site, dtype=bool)
            return is_fascicular_site
        
    # Use k-means clustering to separate the two disks
    centroids, labels = kmeans2(fascicular_points, k=2, minit='points')
    
    # Determine which centroid is closer to LV
    # Calculate vectors from RV center to each centroid
    vectors_to_centroids = centroids - RV_center
    
    # Project these vectors onto the septal normal
    projections = np.dot(vectors_to_centroids, septal_normal)
    
    # The centroid with larger projection is closer to LV
    lv_cluster_label = np.argmax(projections)
    
    # Count points in each cluster
    cluster0_count = np.sum(labels == 0)
    cluster1_count = np.sum(labels == 1)
    print(f"Cluster 0: {cluster0_count} points, Cluster 1: {cluster1_count} points")
    print(f"Selected LV cluster (label {lv_cluster_label}) with {np.sum(labels == lv_cluster_label)} points")
    
    # Create a new boolean array with only the LV cluster points
    is_lv_fascicular_site = np.zeros_like(is_fascicular_site, dtype=bool)
    lv_indices = fascicular_indices[labels == lv_cluster_label]
    is_lv_fascicular_site[lv_indices] = True
    
    return is_lv_fascicular_site

disk_height_tolerance = 0.05
disk_radius = 0.02

coord_t = 0.65
coord_l = 0.30
coord_c = 0.8

is_fascicular_site = np.logical_and( np.abs(phi_transmural - coord_t) < disk_height_tolerance , ((phi_longitudinal - coord_l)**2.0 + (phi_circumferential - coord_c)**2.0) < disk_radius**2.0 ) 

LV_points = np.array(list(set(triangles[triangle_regions == 3].flatten()))) # Left ventricular endocardium
RV_points = np.array(list(set(triangles[triangle_regions == 4].flatten()))) # Right ventricular endocardium

LV_center = points[LV_points].mean(axis=0)
RV_center = points[RV_points].mean(axis=0)

septal_normal = RV_center - LV_center
septal_normal /= np.linalg.norm(septal_normal)

is_fascicular_site = filter_fascicular_sites_closer_to_LV(points, is_fascicular_site, septal_normal, LV_center, RV_center)

Total fascicular sites: 10
Cluster 0: 4 points, Cluster 1: 6 points
Selected LV cluster (label 1) with 6 points


In [27]:
import numpy as np
import pyvista as pv
from pyvista import themes
import os
import random

def visualize_fascicular_sites(points, triangles, triangle_regions, is_fascicular_site, 
                               sphere_scale=0.005):
    """
    Visualize fascicular sites on the endocardial surface.
    
    Parameters:
    -----------
    points : numpy.ndarray
        Array of 3D coordinates of the mesh vertices
    triangles : numpy.ndarray
        Array of triangle indices (each row contains 3 indices)
    triangle_regions : numpy.ndarray
        Array indicating the region each triangle belongs to
        (2 for epicardium, 3 or 4 for endocardium)
    is_fascicular_site : numpy.ndarray
        Boolean array indicating whether each point is a fascicular site
    subsample_factor : int, optional
        Factor by which to subsample points for visualization
    sphere_scale : float, optional
        Scale factor for the spheres representing fascicular sites
    """
    # Set up a nice theme for visualization
    my_theme = themes.DarkTheme()
    my_theme.lighting = True
    my_theme.show_edges = False
    my_theme.background = 'white'
    my_theme.window_size = [1000, 800]
    pv.set_plot_theme(my_theme)
    
    print(f"Original points: {len(points)}")
    print(f"Original triangles: {len(triangles)}")
    print(f"Number of fascicular sites: {np.sum(is_fascicular_site)}")
    
    # Filter triangles to get only endocardium
    endocardium_triangles = triangles[np.logical_or(triangle_regions == 3, triangle_regions == 4)]
    print(f"Endocardium triangles: {len(endocardium_triangles)}")
    
    # Create endocardium mesh
    endocardium_cells = np.column_stack(
        (np.full(len(endocardium_triangles), 3), endocardium_triangles)
    ).flatten()
    
    endocardium_mesh = pv.PolyData(points, endocardium_cells)
    
    # Get indices of fascicular sites - use ALL of them, no subsampling
    fascicular_indices = np.where(is_fascicular_site)[0]
    print(f"Displaying all {len(fascicular_indices)} fascicular sites")
    
    # Create a new point cloud for fascicular sites
    fascicular_points = points[fascicular_indices]
    fascicular_cloud = pv.PolyData(fascicular_points)
    
    # Create a plotter
    plotter = pv.Plotter()
    plotter.add_text("Endocardium with Fascicular Sites", font_size=16, color="black")
    
    # Add endocardium surface
    plotter.add_mesh(endocardium_mesh, color='mistyrose', opacity=0.7, 
                     label="Endocardial Surface")
    
    # Add spheres for fascicular sites
    # Use glyph to create spheres at fascicular sites
    spheres = fascicular_cloud.glyph(
        geom=pv.Sphere(radius=1),
        scale=False,
        factor=sphere_scale
    )
    plotter.add_mesh(spheres, color="red", label="Fascicular Sites")
    
    # Add legend
    plotter.add_legend(size=(0.2, 0.2), loc='upper right')
    
    # Set camera position for a good view
    plotter.camera_position = 'xz'
    plotter.camera.zoom(1.2)

    # Show the plot
    plotter.show()

    plotter.screenshot('../figures/fascicular_sites.png') 
    
    return plotter

# Example usage:
visualize_fascicular_sites(
    points, triangles, triangle_regions,
    is_fascicular_site, sphere_scale=1
)

Original points: 110822
Original triangles: 77128
Number of fascicular sites: 6
Endocardium triangles: 39581
Displaying all 6 fascicular sites


/Users/jamesmcgreivy/miniconda3/envs/openCARP/lib/python3.10/site-packages/pyvista/core/filters/data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:63147/index.html?ui=P_0x3222ea0b0_13&reconnect=auto" class="pyvist…

In [28]:
import numpy as np
import os

def save_fascicular_sites_to_vtx(is_fascicular_site, output_filename):
    """
    Save the indices of fascicular sites to a .vtx file.
    """
    # Get indices of fascicular sites
    site_indices = np.where(is_fascicular_site)[0]
    num_sites = len(site_indices)
    
    print(f"Saving {num_sites} fascicular site indices to {output_filename}")
    
    # Write to file
    with open(output_filename, 'w') as f:
        # Write header (number of sites)
        f.write(f"{num_sites}\n")
        f.write("extra\n")
        
        # Write each index on a new line
        for idx in site_indices:
            f.write(f"{idx}\n")
    
    print(f"Successfully saved fascicular sites to {os.path.abspath(output_filename)}")
    return os.path.abspath(output_filename)

# Save to file:
save_fascicular_sites_to_vtx(is_fascicular_site, output_filename=f"{data_path}fascicular_stim.vtx")

Saving 6 fascicular site indices to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/fascicular_stim.vtx
Successfully saved fascicular sites to /Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/fascicular_stim.vtx


'/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/fascicular_stim.vtx'

##  Extra functions

In [21]:
import numpy as np
import pyvista as pv
import random

def visualize_phi(points, phi, subsample_factor=10, point_size=5, cmap="viridis"):
    """
    Visualize a scalar field using just points (no mesh connectivity required).
    """
    # Set up theme
    pv.set_plot_theme("document")
    
    print(f"Original points: {len(points)}")
    
    # Subsample points to make visualization manageable
    n_points = len(points)
    n_sample = n_points // subsample_factor
    
    # Make sure we don't have too few or too many points
    n_sample = max(1000, min(n_sample, 50000))
    
    print(f"Subsampling points from {n_points} to {n_sample}")
    
    # Random sampling
    sample_indices = sorted(random.sample(range(n_points), n_sample))
    
    # Create subsampled arrays
    sampled_points = points[sample_indices]
    sampled_phi = phi[sample_indices]
    
    # Create point cloud
    cloud = pv.PolyData(sampled_points)
    cloud.point_data["transmural"] = sampled_phi
    
    # Create plotter
    plotter = pv.Plotter()
    plotter.add_text("Phi Point Cloud", font_size=14)
    
    # Add point cloud with scalar values
    plotter.add_mesh(cloud, render_points_as_spheres=True, point_size=point_size,
                    scalars="transmural", cmap=cmap, opacity=1.0,
                    show_scalar_bar=True, scalar_bar_args={"title": "Phi"})
    
    # Set view and show
    plotter.view_isometric()
    plotter.show()

visualize_phi(points, phi_longitudinal, subsample_factor=1, point_size=5)

Original points: 110822
Subsampling points from 110822 to 50000


Widget(value='<iframe src="http://localhost:63147/index.html?ui=P_0x3105337f0_10&reconnect=auto" class="pyvist…

In [ ]:
import numpy as np

def read_fiber_directions(lon_file_path):
    """
    Read fiber directions from a .lon file
    
    Parameters:
    -----------
    lon_file_path : str
        Path to the .lon file
        
    Returns:
    --------
    numpy.ndarray
        Array of shape (n_elements, 3) containing the fiber directions
    """
    # Open the file and read all lines
    with open(lon_file_path, 'r') as f:
        lines = f.readlines()
    
    # Skip the first line (header)
    data_lines = lines[1:]
    
    # Initialize an array to store fiber directions
    n_elements = len(data_lines)
    fiber_directions = np.zeros((n_elements, 3))
    
    # Parse each line to extract the fiber directions
    for i, line in enumerate(data_lines):
        # Split the line into values
        values = line.strip().split()
        
        # First 3 values are the fiber direction
        fiber_directions[i, 0] = float(values[0])
        fiber_directions[i, 1] = float(values[1])
        fiber_directions[i, 2] = float(values[2])
    
    return fiber_directions

def map_element_fibers_to_points(element_fiber_dirs, tetrahedra, n_points):
    """
    Map element fiber directions to mesh points using a vectorized approach
    
    Parameters:
    -----------
    element_fiber_dirs : numpy.ndarray
        Array of shape (n_elements, 3) containing fiber directions for each element
    tetrahedra : numpy.ndarray
        Array of shape (n_elements, 4) containing point indices for each tetrahedron
    n_points : int
        Total number of points in the mesh
        
    Returns:
    --------
    numpy.ndarray
        Array of shape (n_points, 3) containing averaged fiber directions for each point
    """
    # Initialize arrays to store sum of fiber directions and count of elements per point
    fiber_sum = np.zeros((n_points, 3))
    count = np.zeros(n_points)
    
    # Create a flattened index of point-element pairs
    elem_indices = np.repeat(np.arange(len(tetrahedra)), 4)
    point_indices = tetrahedra.flatten()
    
    # Add the fiber direction of each element to its associated points
    for i, p_idx in enumerate(point_indices):
        elem_idx = elem_indices[i]
        fiber_sum[p_idx] += element_fiber_dirs[elem_idx]
        count[p_idx] += 1
    
    # Avoid division by zero
    count[count == 0] = 1
    
    # Compute average fiber direction for each point
    fiber_dirs = fiber_sum / count[:, np.newaxis]
    
    # Normalize the fiber directions
    norms = np.linalg.norm(fiber_dirs, axis=1)
    norms[norms == 0] = 1  # Avoid division by zero
    fiber_dirs = fiber_dirs / norms[:, np.newaxis]
    
    return fiber_dirs

# Usage example
lon_file_path = "/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001_lowres/heart_instance_001_lowres.lon"
element_fiber_dirs = read_fiber_directions(lon_file_path)

# Assuming points and tetrahedra are already defined
n_points = len(points)
fiber_dirs = map_element_fibers_to_points(element_fiber_dirs, tetrahedra, n_points)

In [ ]:
import vtk
import numpy as np
import pandas as pd
from vtk.util import numpy_support
import os

# Function to read VTP files
def read_vtp(filename):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(filename)
    reader.Update()
    polydata = reader.GetOutput()
    return polydata

# Function to read VTU files
def read_vtu(filename):
    reader = vtk.vtkXMLUnstructuredGridReader()
    reader.SetFileName(filename)
    reader.Update()
    grid = reader.GetOutput()
    return grid

# Function to extract and save Universal Ventricular Coordinates (UVC)
def extract_save_uvc(volume_mesh, output_prefix, output_path):
    # UVC coordinate names to extract
    uvc_names = ["tv", "tm", "rtSin", "rtCos", "rt", "ab"]
    
    # Dictionary to store the coordinates
    coordinates = {}
    
    # Extract each coordinate if available
    for coord_name in uvc_names:
        if volume_mesh.GetPointData().HasArray(coord_name):
            data_array = volume_mesh.GetPointData().GetArray(coord_name)
            coordinates[coord_name] = numpy_support.vtk_to_numpy(data_array)
            print(f"Extracted {coord_name} shape: {coordinates[coord_name].shape}")
            print(f"{coord_name} range: {coordinates[coord_name].min()} to {coordinates[coord_name].max()}")
    
    # Create a pandas DataFrame from the coordinates
    if coordinates:
        df = pd.DataFrame(coordinates)
        # Create full path for outputs
        csv_path = os.path.join(output_path, f"UVC_{output_prefix}.csv")
        # Save to CSV
        df.to_csv(csv_path, index=False)
        print(f"Saved UVC data to {csv_path}")
    else:
        print("No UVC data found in the volume mesh")

# Function to convert VTK data to openCARP format
def convert_to_opencarp(volume_mesh, surface_mesh, output_prefix, output_path):
    # Extract and save UVC data
    extract_save_uvc(volume_mesh, output_prefix, output_path)
    
    # Generate full paths for output files
    pts_path = os.path.join(output_path, f"{output_prefix}.pts")
    elem_path = os.path.join(output_path, f"{output_prefix}.elem")
    surf_path = os.path.join(output_path, f"{output_prefix}.surf")
    
    # Generate .pts file from volume mesh points
    points = volume_mesh.GetPoints()
    num_points = points.GetNumberOfPoints()
    
    with open(pts_path, "w") as pts_file:
        pts_file.write(f"{num_points}\n")
        for i in range(num_points):
            point = points.GetPoint(i)
            pts_file.write(f"{point[0]} {point[1]} {point[2]}\n")
    
    print(f"Created {pts_path} with {num_points} points")
    
    # Generate .elem file from volume mesh cells (tetrahedra)
    num_cells = volume_mesh.GetNumberOfCells()
    
    with open(elem_path, "w") as elem_file:
        elem_file.write(f"{num_cells}\n")
        
        # Determine if we have cell data for regions
        cell_data = None
        region_id_array_name = None
        
        for i in range(volume_mesh.GetCellData().GetNumberOfArrays()):
            array_name = volume_mesh.GetCellData().GetArrayName(i)
            # Possible names for region IDs in the cell data
            if array_name in ["region", "material", "celldata"]:
                region_id_array_name = array_name
                cell_data = numpy_support.vtk_to_numpy(volume_mesh.GetCellData().GetArray(array_name))
                break
        
        # Default region ID if no cell data is available
        default_region_id = 0
        
        for i in range(num_cells):
            cell = volume_mesh.GetCell(i)
            
            # Check if the cell is a tetrahedron (VTK_TETRA = 10)
            if cell.GetCellType() == 10:  # VTK_TETRA
                # Get the 4 point IDs of the tetrahedron
                point_ids = [cell.GetPointId(j) for j in range(4)]
                
                # Get region ID for this cell if available
                region_id = default_region_id
                if cell_data is not None:
                    region_id = int(cell_data[i])
                
                # Write in Tt format for tetrahedron with region ID
                # Note: openCARP is 0-indexed, same as VTK
                elem_file.write(f"Tt {point_ids[0]} {point_ids[1]} {point_ids[2]} {point_ids[3]} {region_id}\n")
            else:
                print(f"Warning: Ignoring non-tetrahedral cell (type {cell.GetCellType()}) at index {i}")
    
    print(f"Created {output_prefix}.elem with tetrahedral elements")
    
    # Generate .surf file from surface mesh triangles
    # Extract class/region information if available
    region_labels = None
    if surface_mesh.GetPointData().HasArray("class"):
        class_array = surface_mesh.GetPointData().GetArray("class")
        region_labels = numpy_support.vtk_to_numpy(class_array)
    
    # Create a dictionary to organize triangles by region
    triangles_by_region = {}
    
    num_surface_cells = surface_mesh.GetNumberOfCells()
    for i in range(num_surface_cells):
        cell = surface_mesh.GetCell(i)
        
        # Check if the cell is a triangle (VTK_TRIANGLE = 5)
        if cell.GetCellType() == 5:  # VTK_TRIANGLE
            # Get the 3 point IDs of the triangle
            point_ids = [cell.GetPointId(j) for j in range(3)]
            
            # Determine region for this triangle
            # We need to map the surface point IDs to volume point IDs
            # For simplicity, we'll use the most common region among the triangle's points
            region = 0  # Default region
            
            if region_labels is not None:
                # Get the region labels for the points in this triangle
                triangle_regions = [region_labels[point_id] for point_id in point_ids]
                
                # Use the most common region label for this triangle
                from collections import Counter
                region = Counter(triangle_regions).most_common(1)[0][0]
            
            # Add this triangle to the appropriate region
            if region not in triangles_by_region:
                triangles_by_region[region] = []
            
            triangles_by_region[region].append(point_ids)
    
    # Write the .surf file with triangles organized by region
    with open(surf_path, "w") as surf_file:
        # For each region, write the number of triangles in that region followed by Reg ID
        for region, triangles in sorted(triangles_by_region.items()):
            num_triangles_in_region = len(triangles)
            surf_file.write(f"{num_triangles_in_region} Reg {region}\n")
            
            # Write all triangles for this region
            for point_ids in triangles:
                surf_file.write(f"Tr {point_ids[0]} {point_ids[1]} {point_ids[2]}\n")
    
    print(f"Created {output_prefix}.surf with surface triangles")

# Main function to read VTK files and convert to openCARP format
def vtk_to_opencarp(vtp_filename, vtu_filename, output_prefix, output_path):
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    print(f"Reading surface mesh: {vtp_filename}")
    surface_mesh = read_vtp(vtp_filename)
    
    print(f"Reading volume mesh: {vtu_filename}")
    volume_mesh = read_vtu(vtu_filename)
    
    print(f"Converting to openCARP format. Output directory: {output_path}")
    convert_to_opencarp(volume_mesh, surface_mesh, output_prefix, output_path)
    
    print(f"Conversion complete! Files saved to: {output_path}")


data_path = "/Users/jamesmcgreivy/Desktop/opencarp_test/full-heart-simulation/data/instance_001/"
vtp_path = data_path + "instance_001.vtp"
vtu_path = data_path + "instance_001.vtu"
output_prefix = "heart_instance_001"

vtk_to_opencarp(vtp_path, vtu_path, output_prefix, data_path)